In [1]:
import numpy as np
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv
load_dotenv()


/anaconda/envs/ml-gpu/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [2]:
fname = './data/function_calls.json'
import json
with open(fname) as f:
    data = json.load(f)



In [3]:
from langchain_core.prompts import ChatPromptTemplate
import os 

llm = AzureChatOpenAI(
                azure_endpoint = os.getenv("AZURE_ENDPOINT"),
                openai_api_key = os.getenv("OPENAI_API_KEY"),
                deployment_name=os.getenv("DEPLOYMENT_NAME"),  # or your deployment
                openai_api_version=os.getenv("OPENAI_API_VERSION"),  # or your api version
                temperature=0.7,
                max_tokens=None,
                timeout=None,
                max_retries=2,
            )


def generate_command(function_call, n = 5):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''You are a car driving command generator that generated human like commands to execute a given function. You have been given a function execution string like function_name(parameter_name1=parameter_value1, parameter_name2=parameter_value2, ...).\n
                                        You need to write a user command that would cause the function to be called.\n
                                        Assume that user is asking while driving a car and is using voice commands.\n
                                        You can be authoritative or commanding or polite while writing the command.\n
                                        '''
            ),
            ("human", "{function_call}"),
        ]
    )

    chain = prompt | llm 
    
    all_commands = []
    
    for i in range(n):
        res = chain.invoke(
            {
                "function_call": function_call
            }
        )
        
        all_commands.append(res.content)
    
    return all_commands

In [4]:
import time
for function in data.keys():
    print("Generating command for function: ", function)
    data[function]['user_commands'] = []
    for i,call in enumerate(data[function]['calls']):
        
        print("Generating command for call: ", call)
        data[function]['user_commands'].append(generate_command(call, n=2))

        if i % 10 == 0 :
            # wait for 1 second
            time.sleep(1)
    
    
        
        
    

Generating command for function:  adjust_temperature
Generating command for call:  adjust_temperature(temperature=14)
Generating command for call:  adjust_temperature(temperature=34)
Generating command for call:  adjust_temperature(temperature=57, zone=front)
Generating command for call:  adjust_temperature(temperature=70, zone=front)
Generating command for call:  adjust_temperature(temperature=8, zone=all)
Generating command for call:  adjust_temperature(temperature=59, zone=rear)
Generating command for call:  adjust_temperature(temperature=53)
Generating command for call:  adjust_temperature(temperature=30, zone=all)
Generating command for call:  adjust_temperature(temperature=25)
Generating command for call:  adjust_temperature(temperature=48, zone=rear)
Generating command for call:  adjust_temperature(temperature=6)
Generating command for call:  adjust_temperature(temperature=29)
Generating command for call:  adjust_temperature(temperature=10, zone=rear)
Generating command for call

In [5]:
fname = './data/function_calls_with_commands.json'
with open(fname, 'w') as f:
    json.dump(data, f, indent=4)